In [2]:
from __future__ import print_function, division
%load_ext autoreload

In [3]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

In [4]:
dirname = os.getcwd()
DATA_FILEPATH     = os.path.join(dirname, "..", "data", "all_hourly_data.h5")
RAW_DATA_FILEPATH = os.path.join(dirname, "..", "data", "all_hourly_data.h5")
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '0'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out

class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [6]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs')
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

Wall time: 28 s


In [7]:
pd.set_option('display.max_columns', 20)

In [11]:
data_full_lvl2.head(100)

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   
...                                                         ...   ...  ...   
                              95                            0.0   NaN  NaN   
                              96                            0.0   NaN  NaN   
                              97                            0.0   NaN  NaN   
                              98                            0.0   NaN  NaN   
                              99                            0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   
...                                        ...  ...  ...             ...  ...   
                              95           0.0  NaN  NaN             0.0  NaN   
                              96           0.0  NaN  NaN             0.0  NaN   
                              97           0.0  NaN  NaN             0.0  NaN   
                              98           0.0  NaN  NaN             0.0  NaN   
                              99           0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   
...                                     ..             ...  ...   
                              95       NaN             0.0  ...   
                              96       NaN             0.0  ...   
                              97       NaN             0.0  ...   
                              98       NaN             0.0  ...   
                              99       NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   
...                                                       ...   
                              95                          NaN   
                              96                          NaN   
                              97                          NaN   
                              98                     

In [25]:
data_full_raw.head().T

subject_id                                             3                     
hadm_id                                           145834                     
icustay_id                                        211552                     
hours_in                                               0    1     2    3    4
LEVEL2                   Aggregation Function                                
alanine aminotransferase count                  2.000000  0.0  0.00  0.0  0.0
                         mean                  25.000000  NaN   NaN  NaN  NaN
                         std                    0.000000  NaN   NaN  NaN  NaN
albumin                  count                  2.000000  0.0  0.00  0.0  0.0
                         mean                   1.800000  NaN   NaN  NaN  NaN
...                                                  ...  ...   ...  ...  ...
ph                       mean                   7.400000  NaN  7.26  NaN  NaN
                         std                    0.147733  NaN  0.00  NaN  NaN
ph urine                 count                  1.000000  0.0  0.00  0.0  0.0
                         mean                   5.000000  NaN   NaN  NaN  NaN
                         std                         NaN  NaN   NaN  NaN  NaN

[312 rows x 5 columns]

In [18]:
statics.head().T

subject_id,3,4,6,9,11
hadm_id,145834,185777,107064,150750,194540
icustay_id,211552,294638,228232,220597,229441
gender,M,F,F,M,F
ethnicity,WHITE,WHITE,WHITE,UNKNOWN/NOT SPECIFIED,WHITE
age,76.5268,47.845,65.9423,41.7902,50.1483
insurance,Medicare,Private,Medicare,Medicaid,Private
admittime,2101-10-20 19:08:00,2191-03-16 00:28:00,2175-05-30 07:15:00,2149-11-09 13:06:00,2178-04-16 06:18:00
diagnosis_at_admission,HYPOTENSION,"FEVER,DEHYDRATION,FAILURE TO THRIVE",CHRONIC RENAL FAILURE/SDA,HEMORRHAGIC CVA,BRAIN MASS
dischtime,2101-10-31 13:58:00,2191-03-23 18:41:00,2175-06-15 16:00:00,2149-11-14 10:15:00,2178-05-11 19:00:00
discharge_location,SNF,HOME WITH HOME IV PROVIDR,HOME HEALTH CARE,DEAD/EXPIRED,HOME HEALTH CARE


In [9]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2, raw = [df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE)
] for df in (data_full_lvl2, data_full_raw)]

# raw.columns = raw.columns.droplevel(level=['label', 'LEVEL1', 'LEVEL2'])

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, raw_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, raw, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (raw_train, raw_dev, raw_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2, raw, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)
raw_means, raw_stds = raw_train.loc[:, idx[:,'mean']].mean(axis=0), raw_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_dev.loc[:, idx[:,'mean']] = (lvl2_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_test.loc[:, idx[:,'mean']] = (lvl2_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

raw_train.loc[:, idx[:,'mean']] = (raw_train.loc[:, idx[:,'mean']] - raw_means)/raw_stds
raw_dev.loc[:, idx[:,'mean']] = (raw_dev.loc[:, idx[:,'mean']] - raw_means)/raw_stds
raw_test.loc[:, idx[:,'mean']] = (raw_test.loc[:, idx[:,'mean']] - raw_means)/raw_stds

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [8]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [10]:
raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test = [
    simple_imputer(df) for df in (raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test)
]
raw_flat_train, raw_flat_dev, raw_flat_test, lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test
    )
]

for df in lvl2_train, lvl2_dev, lvl2_test, raw_train, raw_dev, raw_test: assert not df.isnull().any().any()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

### Task Prediction

#### Hyperparams

In [11]:
N = 15

LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3, 10000)),
    'penalty': Choice(['l1', 'l2']),
    'solver': Choice(['liblinear', 'lbfgs']),
    'max_iter': Choice([100, 500])
})
np.random.seed(SEED)
LR_hyperparams_list = LR_dist.rvs(N)
for i in range(N):
    if LR_hyperparams_list[i]['solver'] == 'lbfgs': LR_hyperparams_list[i]['penalty'] = 'l2'

RF_dist = DictDist({
    'n_estimators': ss.randint(50, 500),
    'max_depth': ss.randint(2, 10),
    'min_samples_split': ss.randint(2, 75),
    'min_samples_leaf': ss.randint(1, 50),
})
np.random.seed(SEED)
RF_hyperparams_list = RF_dist.rvs(N)

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95), 
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 7),
    'batch_size': ss.randint(35, 65),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

In [12]:
def run_basic(model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, target):
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))
        M = model(**hyperparams)
        M.fit(X_flat_train, Ys_train[target])
        s = roc_auc_score(Ys_dev[target], M.predict_proba(X_flat_dev)[:, 1])
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams
            print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))

    return run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target)

def run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target):
    best_M = model(**best_hyperparams)
    best_M.fit(pd.concat((X_flat_train, X_flat_dev)), pd.concat((Ys_train, Ys_dev))[target])
    y_true  = Ys_test[target]
    y_score = best_M.predict_proba(X_flat_test)[:, 1]
    y_pred  = best_M.predict(X_flat_test)

    auc   = roc_auc_score(y_true, y_score)
    auprc = average_precision_score(y_true, y_score)
    acc   = accuracy_score(y_true, y_pred)
    F1    = f1_score(y_true, y_pred)
    
    return best_M, best_hyperparams, auc, auprc, acc, F1

### Sklearn

In [41]:
RESULTS_PATH = '/pickles/extraction_baselines-sklearn.pkl'
with open(RESULTS_PATH, mode='rb') as f: results = pickle.load(f)
    
RERUN = True

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/mmd/extraction_baselines-sklearn.pkl'

In [16]:
results = {}
RERUN = False

In [17]:
for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list), ('LR', LogisticRegression, LR_hyperparams_list)
]:
    if model_name not in results: results[model_name] = {}
    for t in ['los_7', 'los_3']:
        if t not in results[model_name]: results[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            # ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results[model_name][t][n][1]
                    results[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results[model_name][t][n][2:])

                    with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(results[model_name][t][n][2:])  # auc, auprc, acc, F1
            # with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
            

Running model RF on target mort_icu with representation lvl2
On sample 1 / 15 (hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8})
New Best Score: 83.95 @ hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8}
On sample 2 / 15 (hyperparams = {'n_estimators': 285, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 4})
New Best Score: 85.19 @ hyperparams = {'n_estimators': 285, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 4}
On sample 3 / 15 (hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7})
New Best Score: 86.80 @ hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7}
On sample 4 / 15 (hyperparams = {'n_estimators': 122, 'max_depth': 8, 'min_samples_split': 65, 'min_samples_leaf': 22})
New Best Score: 87.53 @ hyperparams = {'n_estimators': 122, 'max_depth': 8, 'min_samples_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 3 / 15 (hyperparams = {'C': 0.003491839757169992, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 4 / 15 (hyperparams = {'C': 48.783036208459954, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100})


...

On sample 10 / 15 (hyperparams = {'C': 0.11951096159304532, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 11 / 15 (hyperparams = {'C': 433.47464177487655, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
New Best Score: 67.25 @ hyperparams = {'C': 433.47464177487655, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500}
On sample 12 / 15 (hyperparams = {'C': 1.7295128471299193, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
On sample 13 / 15 (hyperparams = {'C': 1.6777315525707752, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 14 / 15 (hyperparams = {'C': 0.059893230394471704, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 500})
New Best Score: 68.89 @ hyperparams = {'C': 0.059893230394471704, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 500}
On sample 15 / 15 (hyperparams = {'C': 0.032340816118081595, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 100})
New Best Score: 69.23 @ hyperparams = {'C': 0.032340816118081595, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 100}
Final results for model LR on target los_3 with representation raw
(0.6999299890043752, 0.6380930252491623, 0.6674321503131524, 0.5544055944055944)


In [20]:
###### np.random.seed(SEED+1)
LR_hyperparams_list_2 = LR_dist.rvs(45)
for i in range(45):
    if LR_hyperparams_list_2[i]['solver'] == 'lbfgs': LR_hyperparams_list_2[i]['penalty'] = 'l2'

results_2 = {}
results_2_PATH = 'pickles/extraction_baselines-sklearn_LR_2_runs.pkl'

for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list),
    ('LR', LogisticRegression, LR_hyperparams_list_2)
]:
    if model_name not in results_2: results_2[model_name] = {}
    for t in ['los_7', 'los_3']:
        if t not in results_2[model_name]: results_2[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
#             ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results_2[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results_2[model_name][t][n][1]
                    results_2[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results_2[model_name][t][n][2:])

                    with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results_2[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
            print(results_2[model_name][t][n][2:])
            

Running model RF on target los_7 with representation lvl2
On sample 1 / 15 (hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8})
New Best Score: 76.86 @ hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8}
On sample 2 / 15 (hyperparams = {'n_estimators': 285, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 4})
On sample 3 / 15 (hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7})
New Best Score: 77.64 @ hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7}
On sample 4 / 15 (hyperparams = {'n_estimators': 122, 'max_depth': 8, 'min_samples_split': 65, 'min_samples_leaf': 22})
On sample 5 / 15 (hyperparams = {'n_estimators': 305, 'max_depth': 7, 'min_samples_split': 63, 'min_samples_leaf': 4})
On sample 6 / 15 (hyperparams = {'n_estimators': 443, 'max_depth': 4, 'min_samples_split': 24, 'min_sa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


New Best Score: 53.93 @ hyperparams = {'C': 4.901951803445354, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500}
On sample 2 / 45 (hyperparams = {'C': 0.08472133669971818, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


New Best Score: 54.33 @ hyperparams = {'C': 0.08472133669971818, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500}
On sample 3 / 45 (hyperparams = {'C': 167.54150496746925, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
New Best Score: 68.45 @ hyperparams = {'C': 167.54150496746925, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500}
On sample 4 / 45 (hyperparams = {'C': 51.55505174915498, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


...

On sample 43 / 45 (hyperparams = {'C': 0.0020175786941126067, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 44 / 45 (hyperparams = {'C': 0.0030538555088334154, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 45 / 45 (hyperparams = {'C': 16.85864564615854, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
Final results_2 for model LR on target los_3 with representation lvl2
(0.6956139793427512, 0.6282344624693305, 0.6605427974947808, 0.5289687137891077)


In [ ]:
with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)

In [ ]:
for model_name, model, hyperparams_list in [
#     ('RF', RandomForestClassifier, RF_hyperparams_list),
    ('LR', LogisticRegression, LR_hyperparams_list_2)
]:
    if model_name not in results_2: results_2[model_name] = {}
    for t in ['mort_icu', 'los_3']:
        if t not in results_2[model_name]: results_2[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
#             ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            ('raw', raw_flat_train, raw_flat_dev, raw_flat_test),
        ):
            if n in results_2[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results_2[model_name][t][n][1]
                    results_2[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results_2[model_name][t][n][2:])

                    with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results_2[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
            print(results_2[model_name][t][n][2:])
            with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)

In [ ]:
for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list), ('LR', LogisticRegression, LR_hyperparams_list)
]:
    if model_name not in results: results[model_name] = {}
    for t in ['mort_hosp', 'los_7']:
        if t not in results[model_name]: results[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results[model_name][t][n][1]
                    results[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results[model_name][t][n][2:])

                    with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(results[model_name][t][n][2:])
            with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)

In [17]:
torch.cuda.current_device()


AssertionError: Torch not compiled with CUDA enabled

In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"